<a href="https://colab.research.google.com/github/nicolas-errandonea/TFG/blob/TFGcodes/K_fold_Boots_Kelly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 #primero importamos la primera base de datos
from google.colab import drive
from matplotlib import pyplot as plt
import numpy as np 
import pandas as pd
import random
drive.mount('/content/gdrive', force_remount=True)
path2 = "/content/gdrive/My Drive/Boots_kelly/la_liga_definitive.csv"
path = "/content/gdrive/My Drive/Boots_kelly/la_liga_data_1.csv"
df = pd.read_csv(path)
df2 = pd.read_csv(path2)

Mounted at /content/gdrive


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Not connected to a GPU


In [ ]:
from tensorflow import keras
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
import numpy
from sklearn import metrics
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
import statistics
import math
import scipy.optimize as spo
import scipy

In [ ]:
x=[1,2,3,4,5,6,7,8,9,10]
scipy.stats.percentileofscore(x, x[9], 'strict')/100

0.9

Eliminamos todas aquellas columnas que no son variables predictoras de el dataframe que hemos construido anteriormente.

In [ ]:
cols_to_drop = ['season', 'match_name','date','result', 'home_team', 'away_team', 'home_score', 'away_score', 'winner','h_result','d_result','away_result','h_targ_shot','a_targ_shot' ]


df2.drop( columns = cols_to_drop, inplace = True)

In [ ]:
df=df.drop(0)
df2 = df2.iloc[: , 1:]

In [ ]:
def test0(df,df2):
  prediction_cols=['winner']
  x_train = df2[0:1140]      
  y_train = df[0:1140][prediction_cols]
  x_test = df2[1140:]     
  y_test = df[1140:][prediction_cols]
  return x_train,y_train,x_test,y_test

In [ ]:
def test1(df,df2):
  prediction_cols=['winner']
  x_train = pd.concat([df2[0:760],df2[1140:]])
  y_train = pd.concat([df[0:760][prediction_cols],df[1140:][prediction_cols]])
  x_test = df2[760:1140]     
  y_test = df[760:1140][prediction_cols]
  return x_train,y_train,x_test,y_test

In [ ]:
def test2(df,df2):
  prediction_cols=['winner']
  x_train = pd.concat([df2[0:380],df2[760:]])
  y_train = pd.concat([df[0:380][prediction_cols],df[760:][prediction_cols]])
  x_test = df2[380:760]     
  y_test = df[380:760][prediction_cols]
  return x_train,y_train,x_test,y_test

In [ ]:
def test3(df,df2):
  prediction_cols=['winner']
  x_train = df2[380:]      
  y_train = df[380:][prediction_cols]
  x_test = df2[0:380]     
  y_test = df[0:380][prediction_cols]
  return x_train,y_train,x_test,y_test

In [ ]:
def batch_creator(element,z):
  batch=[]
  for j in range(100):
    batch.append(element[j][z])
  return batch

In [ ]:
def  bootstrap_creator( x_train,y_train,x_test,y_test):
  element=[]
  for i in range(100):
    base_modell = RandomForestClassifier(n_estimators = 1388,bootstrap=True ,max_depth=64,max_samples=None,max_features='auto',min_samples_leaf=6,min_samples_split=12,warm_start=False,ccp_alpha=0.005)
    base_modell.fit(x_train, y_train)
    y_pred = base_modell.predict(x_test)
    prob_y_test=base_modell.predict_proba(x_test)
    element.append(prob_y_test)
  real_list=[]
  for z in range(380):
    batch=batch_creator(element,z)
    real_list.append(batch)
  return real_list 

In [ ]:
def f(xyz,p,v):
  import math
  a=xyz[0]
  b=xyz[1]
  c=xyz[2]
  function=p[0]*math.log1p(v[0]*a)+p[1]*math.log1p(v[1]*b)+p[2]*math.log1p(v[2]*c)
  return -function

In [ ]:
def table_creator(df,df2,y_test,prob_y_test,num1):
 if(num1==0):
  data_kelly=df[1140:1520][['h_odd','d_odd','a_odd','h_result','d_result','away_result']]
 if(num1==1):
  data_kelly=df[760:1140][['h_odd','d_odd','a_odd','h_result','d_result','away_result']]
 if(num1==2):
  data_kelly=df[380:760][['h_odd','d_odd','a_odd','h_result','d_result','away_result']]
 if(num1==3):
  data_kelly=df[0:380][['h_odd','d_odd','a_odd','h_result','d_result','away_result']]
 home_prob=[]
 draw_prob=[]
 away_prob=[]
 for j in range(len(y_test.index)):
   home_prob.append(prob_y_test[j][0])
   draw_prob.append(prob_y_test[j][1])
   away_prob.append(prob_y_test[j][2])
 data_kelly['home_prob']=home_prob
 data_kelly['draw_prob']=draw_prob
 data_kelly['away_prob']=away_prob

 h_bet=[]
 d_bet=[]
 a_bet=[]
 for j in data_kelly.index:
   v=[data_kelly.h_odd[j],data_kelly.d_odd[j],data_kelly.a_odd[j]]
   p=[data_kelly.home_prob[j],data_kelly.draw_prob[j],data_kelly.away_prob[j]]
   xyz_start=[1.0,0.0,0.0]
   bnds=((0,1),(0,1),(0,1))
   cons=({'type':'eq','fun':lambda xyz: xyz[0]+xyz[1]+xyz[2]-1})
   result=spo.minimize(f,xyz_start,args=(p,v),options={"disp":True},constraints=cons,bounds=bnds)
   h_bet.append(result.x[0])
   d_bet.append(result.x[1])
   a_bet.append(result.x[2])
 data_kelly['h_bet']=h_bet
 data_kelly['d_bet']=d_bet
 data_kelly['a_bet']=a_bet
 reality=[]
 for i in data_kelly.index:
  prod=[data_kelly.h_odd[i]*data_kelly.h_bet[i],data_kelly.d_odd[i]*data_kelly.d_bet[i],data_kelly.a_odd[i]*data_kelly.a_bet[i]]
  scalar=prod[0]*data_kelly.h_result[i]+prod[1]*data_kelly.d_result[i]+prod[2]*data_kelly.away_result[i]
  reality.append(scalar)
 data_kelly['reality']=reality
 data_kelly=data_kelly.reset_index()
 data_kelly = data_kelly.drop('index', 1)
 return data_kelly

In [ ]:
 def table_creator_bootstrap(real_list,data_kelly):
   param_mean=[]
   param_max=[]
   param_cuant=[]
   for j in range(380):
     cuant_boots=[]
     for z in range(1000):
       n=random.randint(0,99)
       m=random.randint(0,99)
       v=[data_kelly.h_odd[j],data_kelly.d_odd[j],data_kelly.a_odd[j]]
       p=real_list[j][n]
       xyz_start=[1.0,0.0,0.0]
       bnds=((0,1),(0,1),(0,1))
       cons=({'type':'eq','fun':lambda xyz: xyz[0]+xyz[1]+xyz[2]-1})
       result1=spo.minimize(f,xyz_start,args=(p,v),options={"disp":True},constraints=cons,bounds=bnds)
       p=real_list[j][m]
       result2=spo.minimize(f,xyz_start,args=(p,v),options={"disp":True},constraints=cons,bounds=bnds)
       value=(result1.x[0]-result2.x[0])*(result1.x[0]-result2.x[0])+(result1.x[1]-result2.x[1])*(result1.x[1]-result2.x[1])+(result1.x[2]-result2.x[2])*(result1.x[2]-result2.x[2])
       value=100*value
       cuant_boots.append(value)
     def_value=numpy.quantile(cuant_boots,0.8)
     def_value2=numpy.quantile(cuant_boots,0.5)
     def_value3=numpy.quantile(cuant_boots,1)
     param_cuant.append(def_value)
     param_mean.append(def_value2)
     param_max.append(def_value3)
   data_kelly['w_value_quant']=param_cuant
   data_kelly['w_value_max']=param_max
   data_kelly['w_value_mean']=param_mean
   return data_kelly

In [ ]:
def vect_results(money_earned,num1,defin,boot,year,typeof,maxuse,mean,sd,loss):
  if(num1==0):
   year.append('First_season')
  if(num1==1):
   year.append('Second_season')
  if(num1==2):
   year.append('Third_season')
  if(num1==3):
    year.append('Fourth_season')
  if(defin==0):
    maxuse.append('mean')
  if(defin==1):
    maxuse.append('max')
  if(boot==0):
    typeof.append('frac_kelly')
  if(boot==1):
    typeof.append('max_boots')
  if(boot==2):
    typeof.append('mean_boots')
  if(boot==3):
    typeof.append('cuant_boots')
  mean.append(statistics.mean(money_earned))
  fracaso=0
  for z in money_earned:
   if(z<100):
    fracaso=fracaso+1
  loss.append(fracaso/10000)
  sd.append(numpy.std(money_earned))
  return year,typeof,maxuse,mean,sd,loss
  

In [ ]:
def boots_kelly_function_def(data_kelly,num1,year,typeof,maxuse,mean,sd,loss):
 money_earned_max=[]
 money_earned_mean=[]
 money_earned_cuant=[]
 import math
 for z in range(10000):
   kelly=data_kelly.sample(n=350)
   kelly=kelly.reset_index()
   kelly =kelly.drop('index', 1)
   w=100
   w1=100
   w2=100
   for j in kelly.index:
      if(kelly.h_odd[j]*kelly.h_bet[j]*kelly.home_prob[j]+kelly.d_odd[j]*kelly.d_bet[j]*kelly.draw_prob[j]+kelly.a_odd[j]*kelly.a_bet[j]*kelly.away_prob[j]>1.05):

       #if(kelly.w_value_max[j]<numpy.quantile(kelly.w_value_max,0.6)):
         #w=(kelly.reality[j])*w/(40) +w*(1-1/40)
       #if(kelly.w_value_mean[j]<numpy.quantile(kelly.w_value_mean,0.6)):
          #w1=(kelly.reality[j])*w1/(40) +w1*(1-1/40)
      # if(kelly.w_value_quant[j]<numpy.quantile(kelly.w_value_quant,0.6)):
          #w2=(kelly.reality[j])*w2/(40) +w2*(1-1/40)
         p=scipy.stats.percentileofscore(kelly.w_value_max,kelly.w_value_max[j] , 'strict')/100
         p1=scipy.stats.percentileofscore(kelly.w_value_mean,kelly.w_value_mean[j] , 'strict')/100
         p2=scipy.stats.percentileofscore(kelly.w_value_quant,kelly.w_value_quant[j] , 'strict')/100
         w=(kelly.reality[j])*w/(5*(2+p)) +w*(1-1/(5*(2+p)))
         w1=(kelly.reality[j])*w1/(5*(2+p1)) +w1*(1-1/(5*(2+p1)))
         w2=(kelly.reality[j])*w2/(5*(2+p2)) +w2*(1-1/(5*(2+p2))) 
   money_earned_max.append(w)
   money_earned_mean.append(w1)
   money_earned_cuant.append(w2)
 year,typeof,maxuse,mean,sd,loss=vect_results(money_earned_max,num1,0,1,year,typeof,maxuse,mean,sd,loss)
 year,typeof,maxuse,mean,sd,loss=vect_results(money_earned_mean,num1,0,2,year,typeof,maxuse,mean,sd,loss)
 year,typeof,maxuse,mean,sd,loss=vect_results(money_earned_cuant,num1,0,3,year,typeof,maxuse,mean,sd,loss)
 return year,typeof,maxuse,mean,sd,loss


In [ ]:
def boots_kelly_function(data_kelly,num1,year,typeof,maxuse,mean,sd,loss):
 money_earned_max=[]
 money_earned_mean=[]
 money_earned_cuant=[]
 import math
 for z in range(10000):
   kelly=data_kelly.sample(n=350)
   kelly=kelly.reset_index()
   kelly =kelly.drop('index', 1)
   w=100
   w1=100
   w2=100
   for j in kelly.index:
     if(max(kelly.h_odd[j]*kelly.h_bet[j],kelly.d_odd[j]*kelly.d_bet[j],kelly.a_odd[j]*kelly.a_bet[j])>1.05):
        
      #if(kelly.w_value_max[j]<numpy.quantile(kelly.w_value_max,0.6)):
         #w=(kelly.reality[j])*w/(40) +w*(1-1/40)
      #if(kelly.w_value_mean[j]<numpy.quantile(kelly.w_value_mean,0.6)):
         #w1=(kelly.reality[j])*w1/(40) +w1*(1-1/40)
     # if(kelly.w_value_quant[j]<numpy.quantile(kelly.w_value_quant,0.6)):
        # w2=(kelly.reality[j])*w2/(40) +w2*(1-1/40)
         p=scipy.stats.percentileofscore(kelly.w_value_max,kelly.w_value_max[j] , 'strict')/100
         p1=scipy.stats.percentileofscore(kelly.w_value_mean,kelly.w_value_mean[j] , 'strict')/100
         p2=scipy.stats.percentileofscore(kelly.w_value_quant,kelly.w_value_quant[j] , 'strict')/100
         w=(kelly.reality[j])*w/(5*(2+p)) +w*(1-1/(5*(2+p)))
         w1=(kelly.reality[j])*w1/(5*(2+p1)) +w1*(1-1/(5*(2+p1)))
         w2=(kelly.reality[j])*w2/(5*(2+p2)) +w2*(1-1/(5*(2+p2)))  
   money_earned_max.append(w)
   money_earned_mean.append(w1)
   money_earned_cuant.append(w2)
 year,typeof,maxuse,mean,sd,loss=vect_results(money_earned_max,num1,1,1,year,typeof,maxuse,mean,sd,loss)
 year,typeof,maxuse,mean,sd,loss=vect_results(money_earned_mean,num1,1,2,year,typeof,maxuse,mean,sd,loss)
 year,typeof,maxuse,mean,sd,loss=vect_results(money_earned_cuant,num1,1,3,year,typeof,maxuse,mean,sd,loss)
 return year,typeof,maxuse,mean,sd,loss

In [ ]:
def kelly_function_def(data_kelly,num1,year,typeof,maxuse,mean,sd,loss):
 money_earned=[]
 import math
 for z in range(10000):
   kelly=data_kelly.sample(n=350)
   kelly=kelly.reset_index()
   kelly =kelly.drop('index', 1)
   w=100
   for j in kelly.index:
      if(kelly.h_odd[j]*kelly.h_bet[j]*kelly.home_prob[j]+kelly.d_odd[j]*kelly.d_bet[j]*kelly.draw_prob[j]+kelly.a_odd[j]*kelly.a_bet[j]*kelly.away_prob[j]>1.05):
           w=(kelly.reality[j])*w/(10) +w*(1-1/10)
   money_earned.append(w)
 year,typeof,maxuse,mean,sd,loss=vect_results(money_earned,num1,0,0,year,typeof,maxuse,mean,sd,loss)
 return year,typeof,maxuse,mean,sd,loss

In [ ]:
def kelly_function_(data_kelly,num1,year,typeof,maxuse,mean,sd,loss):
 money_earned=[]
 import math
 for z in range(10000):
   kelly=data_kelly.sample(n=350)
   kelly=kelly.reset_index()
   kelly =kelly.drop('index', 1)
   w=100
   for j in kelly.index:
      if(max(kelly.h_odd[j]*kelly.h_bet[j],kelly.d_odd[j]*kelly.d_bet[j],kelly.a_odd[j]*kelly.a_bet[j])>1.05):
           w=(kelly.reality[j])*w/(10) +w*(1-1/10)
   money_earned.append(w)
 year,typeof,maxuse,mean,sd,loss=vect_results(money_earned,num1,1,0,year,typeof,maxuse,mean,sd,loss) 
 return year,typeof,maxuse,mean,sd,loss

In [ ]:
def evaluate(model, x_test, y_test):
  result=(y_test.to_numpy())
  y_pred = model.predict(x_test)
  hits=0
  total=0
  for i in range(len(y_pred)):
    total=total+1
    if((y_pred[i]-result[i])==[0]):
      hits=hits+1
  print(hits/total)
  accuracy=hits/total
  return accuracy

In [ ]:
import random
year=[]
typeof=[]
maxuse=[]
mean=[]
sd=[]
loss=[]

for i in range(4):
 #num1 = random.randint(0, 3)
 num1=i
 if(num1==0):
   x_train,y_train,x_test,y_test=test0(df,df2)
 if(num1==1):
   x_train,y_train,x_test,y_test=test1(df,df2)
 if(num1==2):
   x_train,y_train,x_test,y_test=test2(df,df2)
 if(num1==3):
   x_train,y_train,x_test,y_test=test3(df,df2)
 real_list=bootstrap_creator( x_train,y_train,x_test,y_test)

  
 base_modell = RandomForestClassifier(n_estimators = 1388,bootstrap=True ,max_depth=64,max_samples=None,max_features='auto',min_samples_leaf=6,min_samples_split=12,warm_start=False,ccp_alpha=0.005)
 base_modell.fit(x_train, y_train)
 y_pred = base_modell.predict(x_test)
 prob_y_test=base_modell.predict_proba(x_test)
 base_accuracy = evaluate(base_modell, x_test, y_test)
 print(base_accuracy)
 data_kelly=table_creator(df,df2,y_test,prob_y_test,num1)
 data_kelly=table_creator_bootstrap(real_list,data_kelly)
 year,typeof,maxuse,mean,sd,loss=boots_kelly_function_def(data_kelly,num1,year,typeof,maxuse,mean,sd,loss)
 year,typeof,maxuse,mean,sd,loss=boots_kelly_function(data_kelly,num1,year,typeof,maxuse,mean,sd,loss)
 year,typeof,maxuse,mean,sd,loss=kelly_function_def(data_kelly,num1,year,typeof,maxuse,mean,sd,loss)
 year,typeof,maxuse,mean,sd,loss=kelly_function_(data_kelly,num1,year,typeof,maxuse,mean,sd,loss)




/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vect

0.5605263157894737
0.5605263157894737
Optimization terminated successfully.    (Exit mode 0)
            Current function value: -0.6753068702458254
            Iterations: 5
            Function evaluations: 26
            Gradient evaluations: 5
Optimization terminated successfully.    (Exit mode 0)
            Current function value: -0.6797205516723844
            Iterations: 6
            Function evaluations: 30
            Gradient evaluations: 6
Optimization terminated successfully.    (Exit mode 0)
            Current function value: -0.679800258110915
            Iterations: 5
            Function evaluations: 25
            Gradient evaluations: 5
Optimization terminated successfully.    (Exit mode 0)
            Current function value: -0.6935687508267023
            Iterations: 7
            Function evaluations: 35
            Gradient evaluations: 7
Optimization terminated successfully.    (Exit mode 0)
            Current function value: -0.7146864283641006
            

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:44: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only


Se han truncado las últimas 5000 líneas del flujo de salida.
Optimization terminated successfully.    (Exit mode 0)
            Current function value: -0.7242168409921526
            Iterations: 6
            Function evaluations: 30
            Gradient evaluations: 6
Optimization terminated successfully.    (Exit mode 0)
            Current function value: -0.7283490035472258
            Iterations: 6
            Function evaluations: 30
            Gradient evaluations: 6
Optimization terminated successfully.    (Exit mode 0)
            Current function value: -0.7257918290368688
            Iterations: 6
            Function evaluations: 30
            Gradient evaluations: 6
Optimization terminated successfully.    (Exit mode 0)
            Current function value: -0.7193165808262532
            Iterations: 6
            Function evaluations: 30
            Gradient evaluations: 6
Optimization terminated successfully.    (Exit mode 0)
            Current function value: -0.72636

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed wh

...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed wh

0.55
0.55
Optimization terminated successfully.    (Exit mode 0)
            Current function value: -0.6848050626007296
            Iterations: 6
            Function evaluations: 30
            Gradient evaluations: 6
Optimization terminated successfully.    (Exit mode 0)
            Current function value: -0.7053521235089473
            Iterations: 6
            Function evaluations: 30
            Gradient evaluations: 6
Optimization terminated successfully.    (Exit mode 0)
            Current function value: -0.6760132768077626
            Iterations: 5
            Function evaluations: 26
            Gradient evaluations: 5
Optimization terminated successfully.    (Exit mode 0)
            Current function value: -0.7059090313484248
            Iterations: 6
            Function evaluations: 30
            Gradient evaluations: 6
Optimization terminated successfully.    (Exit mode 0)
            Current function value: -0.6822824339107507
            Iterations: 5
            F

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:44: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only


Se han truncado las últimas 5000 líneas del flujo de salida.
Optimization terminated successfully.    (Exit mode 0)
            Current function value: -0.674335626434212
            Iterations: 5
            Function evaluations: 25
            Gradient evaluations: 5
Optimization terminated successfully.    (Exit mode 0)
            Current function value: -0.6752038501339831
            Iterations: 5
            Function evaluations: 25
            Gradient evaluations: 5
Optimization terminated successfully.    (Exit mode 0)
            Current function value: -0.6748954217487331
            Iterations: 5
            Function evaluations: 25
            Gradient evaluations: 5
Optimization terminated successfully.    (Exit mode 0)
            Current function value: -0.6747117780331637
            Iterations: 5
            Function evaluations: 25
            Gradient evaluations: 5
Optimization terminated successfully.    (Exit mode 0)
            Current function value: -0.675307

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  import sys


In [ ]:
#nos construimos la tabla
tabla=pd.DataFrame(year,columns=['year'])
tabla['type of']=typeof
tabla['max use']=maxuse
tabla['mean']=mean
tabla['sd']=sd
tabla['loss']=loss


In [ ]:
data_kelly

,h_odd,d_odd,a_odd,h_result,d_result,away_result,home_prob,draw_prob,away_prob,h_bet,d_bet,a_bet,reality,w_value_quant,w_value_max,w_value_mean
0,1.37,5.30,8.45,1,0,0,0.639515,0.224428,0.136057,0.572730,0.268476,0.158794,0.784640,0.092652,0.550867,0.038635
1,3.01,3.31,2.46,0,1,0,0.381428,0.348441,0.270130,0.446527,0.409160,0.144313,1.354318,0.130467,0.549434,0.056293
2,3.30,3.71,2.15,0,0,1,0.278797,0.280688,0.440515,0.265078,0.302409,0.432512,0.929902,0.154670,0.603506,0.069505
3,5.10,4.69,1.59,0,0,1,0.260221,0.256694,0.483085,0.334341,0.310269,0.355391,0.565071,0.113444,0.523567,0.046295
4,2.07,3.60,3.60,1,0,0,0.513021,0.271875,0.215103,0.563581,0.275847,0.160572,1.166613,0.090437,0.527620,0.035812
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,3.07,2.95,2.68,0,1,0,0.375107,0.320266,0.304626,0.438636,0.313622,0.247742,0.925185,0.079195,0.425799,0.030939
376,3.37,3.15,2.36,0,0,1,0.370707,0.320692,0.308601,0.458648,0.336879,0.204473,0.482556,0.066527,0.282174,0.030377
377,3.21,2.85,2.66,0,0,1,0.313393,0.310409,0.376198,0.327014,0.282304,0.390682,1.039215,0.075849,0.238356,0.031053
378,2.66,2.92,3.14,1,0,0,0.428584,0.316257,0.255160,0.496880,0.301590,0.201530,1.321701,0.084475,0.339629,0.035109


In [ ]:
tabla

,year,type of,max use,mean,sd,loss
0,First_season,max_boots,mean,72.801250,13.659772,0.9752
1,First_season,mean_boots,mean,79.599312,15.309338,0.9085
2,First_season,cuant_boots,mean,78.475123,15.028004,0.9250
3,First_season,max_boots,max,50.893575,10.340828,0.9999
4,First_season,mean_boots,max,56.944584,11.796156,0.9994
5,First_season,cuant_boots,max,55.801485,11.515105,0.9999
6,First_season,frac_kelly,mean,73.855447,16.585430,0.9410
7,First_season,frac_kelly,max,49.133677,12.120907,0.9997
8,Second_season,max_boots,mean,131.373450,18.562621,0.0525
9,Second_season,mean_boots,mean,140.893772,20.328536,0.0282


In [ ]:
tabla.to_csv('K_fold_boots_variable_function_10_105.csv',index=False)
!cp K_fold_boots_variable_function_10_105.csv "/content/gdrive/My Drive/Boots_kelly/"